The main purpose of this notebook is to construct a clear picture of subscription lifespans by analyzing transaction logs which helps to discover the pattern of churn in our service and facilitates the creation of robust features to train the model.

#### Import Required Libraries

In [2]:
import pandas as pd
import numpy as np

## 1. Access the Raw Data
The dataset we will be working with is from the [KKBox Churn Prediction Challenge](https://www.kaggle.com/competitions/kkbox-churn-prediction-challenge). To access the data, you will need to authenticate with Kaggle and accept the organization's terms. Once authenticated, download the data, unzip it, and then either save it locally or upload it to your preferred storage solution. Alternatively, you can utilize the `data_extract.py` script, to download the data via Kaggle API, which automatically saves the data locally in the `data` folder.

In [ ]:
# extract data from kaggle
%run ../src/data_extract.py

Although the dataset contains various files, we'll focus on the transaction log and member files for our analysis. 


#### Transaction Logs

In [15]:
file_path = '../data/transactions_v2.csv'

# transaction log schema
dtypes = {
    'msno': str,
    'payment_method_id': int,
    'payment_plan_days': int,
    'plan_list_price': int,
    'actual_amount_paid': int,
    'is_auto_renew': int,
    'transaction_date': str, # initailly read as string
    'membership_expire_date': str, # initially read as string
    'is_cancel': int 
}

# read data as csc
trans_df = pd.read_csv(file_path, header=0, dtype=dtypes)

# Convert datetime columns to datetime format
datetime_cols = ['transaction_date', 'membership_expire_date']
for col in datetime_cols:
    trans_df[col] = pd.to_datetime(trans_df[col])

trans_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0


#### Members Info

In [16]:
file_path = '../data/members_v3.csv'

# member data schema
dtypes = {
    'msno': str,
    'city': int,
    'bd': int,
    'gender': str,
    'registered_via': int,
    'registration_init_time': str # initially read as string
}

# read data as csv
member_df = pd.read_csv(file_path, header=0, dtype=dtypes)

# Convert datetime columns to datetime format
datetime_cols = ['registration_init_time']
for col in datetime_cols:
    member_df[col] = pd.to_datetime(member_df[col])

member_df.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,2011-09-11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,2011-09-15
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,2011-09-15
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,2011-09-15


## 2. Construct Subscriber Lifespan Dataset
To construct subscription lifespan, we'll analyze transaction logs. It involves filtering relevant subscription data and tracking join dates, plan changes, and cancellations.

Let's focus on a single customer, msno = `WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8`, analyzing the transaction log associated with this individual in order to develop our logic.

In [17]:
# all transactions for customer msno 'WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8='
df_filtered = trans_df[trans_df['msno'] == 'WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=']
df_filtered.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
3781,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,41,30,149,149,1,2015-02-21,2018-10-29,0
46574,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,41,30,149,149,1,2015-07-25,2020-08-27,0
53850,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,41,30,149,149,1,2015-02-25,2018-12-24,0
96823,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,41,30,149,119,1,2015-01-10,2018-06-02,0
117965,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,41,30,149,149,1,2015-05-22,2019-11-26,0


The transaction log is often with multiple entries on the same transaction date, like `2016-09-25` for this customer. Records with a payment plan days value of zero or less are considered irrelevant. Many entries change the subscription's expiration date. Due to truncation of the time part in the transaction date field, it's difficult to determine the final entry for a given date. We'll assume the expiration date furthest into the future on a given transaction date is the account's expiration date at that time. While this assumption may not be perfect, follow-up transactions on later dates appear to stabilize the expiration date.

In [26]:
filtered_df = trans_df[
    (trans_df['msno'] == 'WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=') &
    (trans_df['payment_plan_days'] > 0)
]

grouped_df = filtered_df.groupby(['msno', 'transaction_date']).agg(
    expires_at=('membership_expire_date', 'max')
).reset_index()

# sort the result
grouped_df.sort_values(by=['msno', 'transaction_date', 'expires_at']).head()

,msno,transaction_date,expires_at
0,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,2015-01-10,2018-06-02
1,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,2015-01-21,2018-07-03
2,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,2015-01-22,2018-08-03
3,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,2015-01-25,2018-09-03
4,WAMleIuG124oDSSdhcZIECwLcHHHbk4or0y6gxkK7t8=,2015-02-10,2018-10-01
